In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [27]:
df = pd.read_csv("train.csv")
print(df.shape)
print(df.columns[:5])

(1961, 558)
Index(['date_id', 'LME_AH_Close', 'LME_CA_Close', 'LME_PB_Close',
       'LME_ZS_Close'],
      dtype='object')


In [28]:

price_keywords = ['OPEN', 'CLOSE', 'HIGH', 'LOW']
volume_keywords = ['VOLUME', 'VOL']
fx_keywords = ['FX_']
settle_keywords = ['SETTLE']

price_cols = [col for col in df.columns if any(k in col.upper() for k in price_keywords)]
volume_cols = [col for col in df.columns if any(k in col.upper() for k in volume_keywords)]
fx_cols = [col for col in df.columns if any(k in col for k in fx_keywords)]  # FX是大写带下划线
settle_cols = [col for col in df.columns if any(k in col.upper() for k in settle_keywords)]

classified_cols = set(price_cols + volume_cols + fx_cols + settle_cols)
other_cols = [col for col in df.columns if col not in classified_cols and col != 'date_id']

print(f"📈 价格类字段数：{len(price_cols)}")
print(f"📊 成交量字段数：{len(volume_cols)}")
print(f"💱 外汇类字段数：{len(fx_cols)}")
print(f"📉 结算价字段数：{len(settle_cols)}")
print(f"❓ 其他未分类字段数：{len(other_cols)}")

print("\n🎯 价格字段示例：", price_cols[:5])
print("🎯 成交量字段示例：", volume_cols[:5])
print("🎯 外汇字段示例：", fx_cols[:5])
print("🎯 结算字段示例：", settle_cols[:5])
print("🎯 其他字段示例：", other_cols[:5])

📈 价格类字段数：414
📊 成交量字段数：101
💱 外汇类字段数：38
📉 结算价字段数：4
❓ 其他未分类字段数：0

🎯 价格字段示例： ['LME_AH_Close', 'LME_CA_Close', 'LME_PB_Close', 'LME_ZS_Close', 'JPX_Gold_Mini_Futures_Open']
🎯 成交量字段示例： ['JPX_Gold_Mini_Futures_Volume', 'JPX_Gold_Rolling-Spot_Futures_Volume', 'JPX_Gold_Standard_Futures_Volume', 'JPX_Platinum_Mini_Futures_Volume', 'JPX_Platinum_Standard_Futures_Volume']
🎯 外汇字段示例： ['FX_AUDJPY', 'FX_AUDUSD', 'FX_CADJPY', 'FX_CHFJPY', 'FX_EURAUD']
🎯 结算字段示例： ['JPX_Gold_Mini_Futures_settlement_price', 'JPX_Gold_Rolling-Spot_Futures_settlement_price', 'JPX_Platinum_Mini_Futures_settlement_price', 'JPX_RSS3_Rubber_Futures_settlement_price']
🎯 其他字段示例： []


In [29]:
total_cells = df.size
total_missing = df.isna().sum().sum()
print(f"总单元格数: {total_cells:,}")
print(f"总缺失数:   {total_missing:,}")
print(f"总体缺失率: {total_missing / total_cells:.2%}")

总单元格数: 1,094,238
总缺失数:   45,054
总体缺失率: 4.12%


In [30]:
miss_cnt = df.isna().sum()
miss_rate = df.isna().mean()

miss_summary = (
    pd.DataFrame({
        "n_missing": miss_cnt,
        "missing_rate": miss_rate,
        "dtype": df.dtypes
    })
    .sort_values("missing_rate", ascending=False)
)

# 仅看有缺失的列（前20个）
miss_summary_nonzero = miss_summary[miss_summary["n_missing"] > 0]
display(miss_summary_nonzero.head(20))

# 有缺失的列数量
print("有缺失的列数：", (miss_cnt > 0).sum())

,n_missing,missing_rate,dtype
US_Stock_GOLD_adj_open,1713,0.873534,float64
US_Stock_GOLD_adj_close,1713,0.873534,float64
US_Stock_GOLD_adj_low,1713,0.873534,float64
US_Stock_GOLD_adj_high,1713,0.873534,float64
US_Stock_GOLD_adj_volume,1713,0.873534,float64
JPX_Gold_Mini_Futures_settlement_price,116,0.059153,float64
JPX_Platinum_Standard_Futures_Close,116,0.059153,float64
JPX_RSS3_Rubber_Futures_Close,116,0.059153,float64
JPX_Gold_Mini_Futures_Volume,116,0.059153,float64
JPX_Gold_Rolling-Spot_Futures_Volume,116,0.059153,float64


有缺失的列数： 519


In [31]:
row_missing_cnt = df.isna().sum(axis=1)
print("含任意缺失的行比例：", (row_missing_cnt > 0).mean())
print("单行最大缺失列数：", int(row_missing_cnt.max()))
print("缺失最多的前5行索引：", row_missing_cnt.sort_values(ascending=False).head(5).index.tolist())

含任意缺失的行比例： 0.8827129015808262
单行最大缺失列数： 515
缺失最多的前5行索引： [1529, 1551, 1892, 1811, 621]


In [32]:
# 保证时间顺序排列（若已有排序可略）
df = df.sort_values("date_id")

# 选出价格类、汇率类、成交量类等时间连续变量（可按你们定义修改）
price_related_cols = [
    col for col in df.columns
    if any(key in col for key in ['GOLD', 'LME', 'JPX', 'US_Stock', 'FX_', 'SETTLE', 'VOLUME'])
]

# 仅对这些列做前向填充 + 后向填充
df[price_related_cols] = df[price_related_cols].ffill()
df[price_related_cols] = df[price_related_cols].bfill()

# 检查是否仍有缺失值（列层面）
remaining_missing = df[price_related_cols].isnull().sum()
print("仍有缺失的字段（如有）：")
print(remaining_missing[remaining_missing > 0])

仍有缺失的字段（如有）：
Series([], dtype: int64)


In [35]:
total_missing = df.isnull().sum().sum()
print(f"✅ 总缺失值数量: {total_missing}")

✅ 总缺失值数量: 0
